# Notebook 06 – Topic Modeling con LDA

En este notebook aplicamos técnicas de Topic Modeling usando Latent Dirichlet Allocation (LDA) para descubrir los temas principales que aparecen en las reviews de productos para bebés. El objetivo es identificar qué aspectos o temas discuten los usuarios en sus reseñas.


Cargo el dataset de reviews ya procesado desde disco. Este dataset contiene las reviews limpias que preparamos en notebooks anteriores, con el texto preprocesado y las etiquetas de sentimiento.


In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("baby_reviews_small_clean.csv")
df.head()


Para aplicar LDA necesito preparar los textos en un formato específico. Primero importo las librerías necesarias de Gensim para crear el diccionario y el corpus, y luego preparo los textos tokenizados. Cada review debe estar representada como una lista de palabras (tokens) para que LDA pueda trabajar con ella.


In [ ]:
import gensim
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Convertir los textos limpios en listas de tokens
texts = df['clean_text'].dropna().apply(lambda x: x.split()).tolist()

print(f"Total de documentos: {len(texts)}")
print(f"Ejemplo de texto tokenizado: {texts[0][:10]}")


Creo un diccionario que asigna un ID único a cada palabra única que aparece en el corpus. Este diccionario es necesario para convertir los textos en una representación numérica que LDA pueda procesar. También filtro palabras que aparecen en muy pocos documentos (menos de 2) o en demasiados (más del 50% de los documentos) porque no aportan información útil para identificar temas.


In [ ]:
# Crear diccionario
dictionary = corpora.Dictionary(texts)

# Filtrar palabras muy raras o muy comunes
dictionary.filter_extremes(no_below=2, no_above=0.5)

print(f"Tamaño del vocabulario después del filtrado: {len(dictionary)}")
print(f"Primeras 10 palabras del diccionario: {list(dictionary.items())[:10]}")


Convierto los textos tokenizados en un formato de "bag of words" (bolsa de palabras) que LDA puede procesar. Cada documento se representa como una lista de tuplas (id_palabra, frecuencia), donde cada tupla indica cuántas veces aparece cada palabra en ese documento. Este formato es más eficiente para el algoritmo LDA.


In [ ]:
# Convertir textos a formato bag-of-words
corpus = [dictionary.doc2bow(text) for text in texts]

print(f"Tamaño del corpus: {len(corpus)}")
print(f"Ejemplo de documento en formato bag-of-words (primeros 10 elementos): {corpus[0][:10]}")


Entreno el modelo LDA con el corpus preparado. LDA es un modelo probabilístico que asume que cada documento es una mezcla de varios temas, y cada tema es una distribución sobre palabras. Configuro el modelo con 5 temas (num_topics=5) y uso parámetros estándar. El modelo aprenderá qué palabras son características de cada tema y qué temas están presentes en cada documento.


In [ ]:
# Entrenar modelo LDA
num_topics = 5

lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    random_state=42,
    passes=10,
    alpha='auto',
    per_word_topics=True
)

print("Modelo LDA entrenado exitosamente")


Visualizo los temas descubiertos por el modelo. Para cada tema, muestro las 10 palabras más importantes (con mayor probabilidad de aparecer en ese tema). Esto me permite entender qué representa cada tema y darle un nombre descriptivo basado en las palabras clave que lo caracterizan.


In [ ]:
# Mostrar los temas y sus palabras más importantes
for idx, topic in lda_model.print_topics(-1, num_words=10):
    print(f"Tema {idx}:")
    print(topic)
    print()


Calculo la coherencia del modelo para evaluar qué tan bien están definidos los temas. La coherencia mide qué tan semánticamente similares son las palabras dentro de cada tema. Un valor más alto indica que las palabras de un tema están más relacionadas entre sí, lo que sugiere que el tema es más coherente y significativo.


In [ ]:
# Calcular coherencia del modelo
coherence_model = CoherenceModel(
    model=lda_model,
    texts=texts,
    dictionary=dictionary,
    coherence='c_v'
)

coherence_score = coherence_model.get_coherence()
print(f"Coherencia del modelo: {coherence_score:.4f}")


Pruebo diferentes números de temas (de 3 a 7) para encontrar el número óptimo. Para acelerar el proceso, uso menos iteraciones (passes=5) y un subconjunto del corpus para la búsqueda de hiperparámetros. Una vez encontrado el mejor número, entrenaré el modelo final con todos los datos y más iteraciones. El número de temas con mayor coherencia suele ser el mejor, ya que indica que los temas están bien definidos y son interpretables.


In [ ]:
# Probar diferentes números de temas (optimizado para velocidad)
# Usamos un subconjunto del corpus y menos iteraciones para la búsqueda
from tqdm import tqdm

coherence_scores = []
num_topics_range = range(3, 8)  # Reducido de 3-8 a 3-7

# Usar un subconjunto más pequeño para la búsqueda (más rápido)
# Tomamos una muestra representativa del corpus
sample_size = min(1000, len(corpus))
corpus_sample = corpus[:sample_size]
texts_sample = texts[:sample_size]

print(f"Buscando número óptimo de temas usando {sample_size} documentos...")
print("Esto tomará menos tiempo que usar todo el corpus.\n")

for num_topics in tqdm(num_topics_range, desc="Probando temas"):
    model = LdaModel(
        corpus=corpus_sample,  # Usar subconjunto para búsqueda
        id2word=dictionary,
        num_topics=num_topics,
        random_state=42,
        passes=5,  # Reducido de 10 a 5 para búsqueda rápida
        alpha='auto',
        iterations=50  # Limitar iteraciones por pass
    )
    
    coherence_model = CoherenceModel(
        model=model,
        texts=texts_sample,  # Usar subconjunto para coherencia
        dictionary=dictionary,
        coherence='c_v'
    )
    
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append(coherence_score)
    print(f"Temas: {num_topics}, Coherencia: {coherence_score:.4f}")


Visualizo los resultados de coherencia para cada número de temas probado. Esto me permite ver gráficamente qué número de temas produce el mejor modelo y tomar una decisión informada sobre cuántos temas usar en el modelo final.

**¿Qué significa la coherencia?**
La coherencia mide qué tan semánticamente relacionadas están las palabras dentro de cada tema. Un valor más alto indica que:
- Las palabras de un tema aparecen juntas frecuentemente en los documentos
- Los temas son más interpretables y tienen sentido semántico
- El modelo está capturando mejor los temas reales en los datos

**Interpretación de la gráfica:**
- **Eje X**: Número de temas probados (de 3 a 7)
- **Eje Y**: Coherencia del modelo (valores más altos = mejor)
- **Punto más alto**: Indica el número óptimo de temas para este dataset

Si la coherencia baja mucho al aumentar los temas, significa que estamos dividiendo demasiado los documentos y creando temas artificiales. Si sube, significa que más temas capturan mejor la estructura de los datos.


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(list(num_topics_range), coherence_scores, marker='o')
plt.xlabel('Número de Temas')
plt.ylabel('Coherencia')
plt.title('Coherencia vs Número de Temas')
plt.grid(True)
plt.show()

# Encontrar el mejor número de temas
best_num_topics = list(num_topics_range)[np.argmax(coherence_scores)]
best_coherence = max(coherence_scores)
print(f"\nMejor número de temas: {best_num_topics} (Coherencia: {best_coherence:.4f})")


Entreno el modelo final con el número óptimo de temas encontrado en el paso anterior, pero esta vez usando TODO el corpus y más iteraciones para obtener el mejor modelo posible. Este será el modelo que usaré para analizar los temas en las reviews y hacer predicciones sobre qué temas están presentes en cada documento.


In [ ]:
# Entrenar modelo final con el mejor número de temas usando TODO el corpus
print(f"Entrenando modelo final con {best_num_topics} temas usando todo el corpus...")
print("Esto puede tardar unos minutos, pero será el modelo definitivo.\n")

final_lda_model = LdaModel(
    corpus=corpus,  # Ahora sí usamos todo el corpus
    id2word=dictionary,
    num_topics=best_num_topics,
    random_state=42,
    passes=10,  # Reducido de 15 a 10 (suficiente para buen modelo)
    alpha='auto',
    per_word_topics=True,
    iterations=100  # Iteraciones por pass
)

# Mostrar los temas del modelo final
print("Temas del modelo final:")
print("=" * 60)
for idx, topic in final_lda_model.print_topics(-1, num_words=10):
    print(f"\nTema {idx}:")
    print(topic)


Para cada documento (review), obtengo la distribución de temas. Esto me dice qué porcentaje de cada tema está presente en cada review. Por ejemplo, una review podría ser 60% tema 0, 30% tema 1 y 10% tema 2. Esto permite entender qué aspectos discute cada usuario en su reseña.


In [ ]:
# Obtener distribución de temas para cada documento
doc_topics = []
for doc_bow in corpus[:10]:  # Mostrar solo los primeros 10 documentos
    topics = final_lda_model.get_document_topics(doc_bow)
    doc_topics.append(topics)
    print(f"Documento {len(doc_topics)-1}:")
    for topic_id, prob in topics:
        print(f"  Tema {topic_id}: {prob:.4f}")
    print()


Creo una visualización interactiva usando pyLDAvis que permite explorar los temas de manera más intuitiva. Esta visualización muestra la relación entre temas y palabras, y permite ver qué documentos están más relacionados con cada tema. Es una herramienta muy útil para interpretar y validar los resultados del modelo LDA.


In [ ]:
# Crear visualización interactiva
try:
    vis = gensimvis.prepare(final_lda_model, corpus, dictionary, sort_topics=False)
    pyLDAvis.display(vis)
except Exception as e:
    print(f"Error al crear visualización: {e}")
    print("Nota: pyLDAvis puede requerir configuración adicional en algunos entornos")


Guardo el modelo entrenado para poder reutilizarlo más tarde sin tener que volver a entrenarlo. Esto es útil si quiero aplicar el modelo a nuevos documentos o hacer análisis adicionales.


In [ ]:
# Guardar modelo y diccionario
final_lda_model.save("lda_baby_reviews.model")
dictionary.save("dictionary_baby_reviews.dict")

print("Modelo y diccionario guardados exitosamente")
